In [0]:
print("hello, hi this is week case study")

hello, hi this is week case study


In [0]:
%pip install great_expectations==0.15.39
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from great_expectations.dataset import SparkDFDataset
from pyspark.sql.functions import *
sas_token = ""
storageaccount = "balustorage"
container = "destination"

spark.conf.set(f"fs.azure.sas.{container}.{storageaccount}.blob.core.windows.net", sas_token)

paths = dbutils.fs.ls(f"wasbs://{container}@{storageaccount}.blob.core.windows.net/raw_data/bronze/")

validation_results = {}

for file in paths:
  df = spark.read.format("csv").option("header", "true").load(f"wasbs://{container}@{storageaccount}.blob.core.windows.net/raw_data/bronze/{file.name}")
  ge_df = SparkDFDataset(df)

  if "holder_id" in df.columns:
        ge_df.expect_column_values_to_not_be_null("holder_id")
        ge_df.expect_column_values_to_be_unique("holder_id")
        df = df.withColumn("holder_id", concat(lit("****"), substring("holder_id", -2, 2)))

  if "holder_code" in df.columns:
        ge_df.expect_column_values_to_not_be_null("holder_code")
        ge_df.expect_column_values_to_be_unique("holder_code")
        df = df.withColumn("holder_code", lit("****"))

  if "policy_id" in df.columns:
        ge_df.expect_column_values_to_not_be_null("policy_id")
        ge_df.expect_column_values_to_be_unique("policy_id")
        df = df.withColumn("policy_id", concat(lit("****"), substring("policy_id", -2, 2)))
  if "holder_id" in df.columns:
        ge_df.expect_column_values_to_not_be_null("holder_id")
        df = df.withColumn("holder_id",concat(lit("****"), substring("holder_id", -2, 2)))
  if "claim_id" in df.columns:
        ge_df.expect_column_values_to_not_be_null("claim_id")
        ge_df.expect_column_values_to_be_unique("claim_id")
        df = df.withColumn("claim_id", lit("****"))

  if "policy_id" in df.columns:
        ge_df.expect_column_values_to_not_be_null("policy_id")
        df = df.withColumn("policy_id", lit("****"))

  if "agent_id" in df.columns:
        ge_df.expect_column_values_to_not_be_null("agent_id")
        ge_df.expect_column_values_to_be_unique("agent_id")
        df = df.withColumn("agent_id", lit("****"))

  if "premium_id" in df.columns:
        ge_df.expect_column_values_to_not_be_null("premium_id")
        ge_df.expect_column_values_to_be_unique("premium_id")
        df = df.withColumn("premium_id", lit("****"))
  if "amount" in df.columns:
        ge_df.expect_column_values_to_not_be_null("amount")

  if "payment_id" in df.columns:
        ge_df.expect_column_values_to_not_be_null("payment_id")
        ge_df.expect_column_values_to_be_unique("payment_id")
        df = df.withColumn("payment_id", concat(lit("****"), substring("payment_id", -2, 2)))

  if "beneficiary_id" in df.columns:
        ge_df.expect_column_values_to_not_be_null("beneficiary_id")
        ge_df.expect_column_values_to_be_unique("beneficiary_id")
        df = df.withColumn("beneficiary_id",concat(lit("****"), substring("beneficiary_id", -2, 2)))

  if "risk_id" in df.columns:
        ge_df.expect_column_values_to_not_be_null("risk_id")
        ge_df.expect_column_values_to_be_unique("risk_id")
        df = df.withColumn("risk_id",concat(lit("****"), substring("risk_id", -2, 2)))

  if "risk_level" in df.columns:
        ge_df.expect_column_values_to_not_be_null("risk_level")

  result = ge_df.validate()
  validation_results[file.name] = result
  df.write.mode("overwrite").format("delta").save(f"wasbs://{container}@{storageaccount}.blob.core.windows.net/processed_data/silver/{file.name}")

# print(validation_results)







In [0]:
df = spark.read.format("delta").load(f"wasbs://{container}@{storageaccount}.blob.core.windows.net/processed_data/silver/holders")
display(df)

holder_id,holder_code,first_name,last_name,dob,gender,email,phone,created_at
*****1,****,john,smith,1985-03-15 00:00:00.0000000,male,john.smith@email.com,9876543210,2025-08-08 06:24:46.9670000
*****2,****,mary,johnson,1990-07-22 00:00:00.0000000,female,mary.johnson@email.com,9123456780,2025-08-08 06:24:46.9670000
*****3,****,robert,brown,1978-11-05 00:00:00.0000000,male,robert.brown@email.com,9988776655,2025-08-08 06:24:46.9670000
*****4,****,linda,davis,1982-01-19 00:00:00.0000000,female,linda.davis@email.com,9011223344,2025-08-08 06:24:46.9670000
*****5,****,david,miller,1995-09-30 00:00:00.0000000,male,david.miller@email.com,9090909090,2025-08-08 06:24:46.9670000


In [0]:
storageaccount = "balustorage"
container = "destination"
silver_path = f"wasbs://{container}@{storageaccount}.blob.core.windows.net/processed_data/silver/policies"
silver_df = spark.read.format("delta").load(silver_path)

gold_df = silver_df.groupBy("holder_id").agg(
    sum("premium_amount").alias("total_premium_amount")
).select("holder_id", "total_premium_amount")

gold_path = f"wasbs://{container}@{storageaccount}.blob.core.windows.net/processed_data/gold/policies"
gold_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(gold_path)

print("Gold layer table created successfully!")

Gold layer table created successfully!


In [0]:
gold_df = spark.read.format("delta").load(f"wasbs://{container}@{storageaccount}.blob.core.windows.net/processed_data/gold/policies")
display(gold_df)

holder_id,total_premium_amount
*****1,27000.0
*****3,11000.0
*****2,20000.0
*****4,14000.0
